In [ ]:
import json
with open("ABP_News_classification.json", "r") as f:
    articles = json.load(f)

In [ ]:
articles.keys()

In [ ]:
articles['business'][23]

In [ ]:
from openai import OpenAI
import random
import time
import re

try:
    with open("ABP_new_query_doc.json", "r") as f:
        query_doc = json.load(f) 
except FileNotFoundError:
    query_doc = {}

def generate_query_for_article(article_text):

    client = OpenAI(
        base_url="https://integrate.api.nvidia.com/v1",
        api_key="XXXXX"
    )
    
    system_message = """
    You are an AI assistant designed to generate relevant questions in Hindi based on news articles. Your task is to generate exactly one concise and well-structured question in Hindi that is directly answerable from the article. If the article is in English, first understand it and then generate the question in Hindi. Ensure proper grammar and vocabulary usage. The question should be enclosed within ##* markers like this:  ##* question ##*.
    """
    
    completion = client.chat.completions.create(
        model="nvidia/llama-3.3-nemotron-super-49b-v1",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": f"Here is a news article:\n\n{article_text}\n\nGenerate one relevant question whose answer is present in this article:"}
        ],
        temperature=0.6,
        top_p=0.95,
        max_tokens=100,
        frequency_penalty=0,
        presence_penalty=0,
        stream=False
    )
    
    return completion.choices[0].message.content.strip()

if __name__ == "__main__":

    rpm_limit = 40
    wait_time = (60.0 / (rpm_limit * 0.95))
    jitter = random.uniform(0.9, 1.1)
    sleep_duration = wait_time * jitter

    with open("ABP_News_classification.json", "r") as f:
        articles = json.load(f)

    for field in articles.keys():

        if field in ['business', 'astro', 'lifestyle', 'gk', 'education', 'agriculture']:
            continue
        count = 0
        q_d = []
        for article in articles[field]:
 
            if count%40 == 0:
                time.sleep(sleep_duration)
            query = generate_query_for_article(article)
            
            pattern = r'##\*(.*?)##\*'
            match = re.search(pattern, query, re.DOTALL)
            if match:
                q_d.append([article, match.group(1).strip() + "?"])
            else:
                q_d.append([article, None])
        
            if count > 2100:
                break
            else:
                count+=1

        query_doc[field] = q_d

        with open("ABP_new_query_doc.json", "w") as f:
            json.dump(query_doc, f, indent=4)
            

In [6]:
import json
with open("Data/Retrieval/ABP_news/ABP_new_query_doc.json", "r") as f:
    query_doc = json.load(f)

for field in query_doc.keys():
    print(f"{field} : {len(query_doc[field])}")

business : 2102
astro : 2102
lifestyle : 2102
gk : 2102
education : 2102
agriculture : 2102
technology : 2102
states : 2102
entertainment : 2102
sports : 2102


In [1]:
import json

with open("new_training_data/train_data_with_hard_negative.jsonl", "r") as f:
    train_data = [json.loads(line) for line in f]

In [ ]:
x = [1 for sample in train_data if sample.get("hard_negative_flag", -1) == 0]

In [12]:
sum(x)

0

In [8]:
train_data[23]

{'id': 'crosssum_english_englishen_174764',
 'source': 'Instructions: Retrieve the most relevant paragraph from a set of options for a given summary. Summary: Work to replace a bridge on the route of a planned artificial river in Oxford will not be completed until 2025.',
 'target': 'Oxfordshire County Council initially planned to repair the A423 Kennington Railway Bridge until inspections in 2019 found it was unfixable. The authority said dealing with a major road, water and electricity, in order to remove the old bridge, would take longer to complete than first planned. The new £40m bridge is on the route of a £150m flood defence scheme. Work on replacing the bridge, which was built in 1965, started in August. Archaeological survey trenches have also been dug in a field close to the A34 at South Hinksey. Once the construction is finished, the Oxford Flood Alleviation project, which has been delayed further due to the bridge works, will eventually see a stream created to the west of O